# Menganalisis risiko peminjam gagal membayar

Menyiapkan laporan untuk bank bagian kredit. Melakukan analysis  pengaruh status perkawinan seorang nasabah dan jumlah anak terhadap probabilitas ketepatan waktu dalam melunasi pinjaman. Dengan data yang sudah di siapkan bank

Laporan Anda akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.



# Langkah-langkah yang akan diambil diantaranya:

1. Memeriksa nilai yang hilang & memperbaikinya dengan metode fillna
2. Memeriksa value unique di setiap kolom untuk memeriksa apakah ada keanehan & memperbaikinya dengan str.lower()
3. Memeriksa apakah ada value yang tidak masuk akal pada setiap kolom & memperbaikinya dengan nilai median/mean (tergantung kondisi value)
4. Membuat kelompok data untuk nilai kategoris dan numerik yang valuenya banyak untuk mempermudah analisis

# Ikhtisar Data

In [1]:
# Memuat semua perpustakaan
import pandas as pd 


In [2]:
# Menggunakan fungsi numpy di seluruh puustaka ini 

import numpy as np


In [3]:
# Membuat variable  untuk penggunaan di jupyter notebook

data = pd.read_csv('credit_scoring_eng.csv')# muat data

In [4]:
# Tahapan untuk mengksplorasi dataset

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita

data.shape

(21525, 12)

In [7]:
# mari menampilkan N baris pertama

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


### Kesimpulan awal


- [days_employed] data float minus menimbulkan pertanyaan bagaimana memproses data pada kolom tersebut 
- Kolom days_employed bernilai negatif dimana seharusnya bekerja tidak mungkin negatif.
- [dob_years] tidak ada informasi lebih detail mengenai dob dimana kemungkinan besar adalah singakatan
- Kolom days_employed jumlah hari kerjanya ada yang tidak masuk akal 340266 hari / 365 hari = 932 tahun.
- banyak bentuk data float yang tidak memiliki detail lebih lanjut


In [6]:
# Mendapatkan informasi data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB




- Nilai yang biasanya di sebut null terlihat tidak memiliki nilai yang hilang terlihat dengan deskrpisi kolom non-null count 

- Oleh karena itu di butuhkan pengecekan

In [8]:
# Mari kita melihat tabel yang telah difilter dengan nilai yang hilang di kolom pertama dengan data yang hilang

data.loc[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


### Kesimpulan Awal 
- data [days_employed] dan [total_income] berubah menjadi nan atau object
- Terlihat nilang yang hilang di kolom days_employed simetris dengan kolom total_income. Mungkin saja terjadi error system pada saat data diinput.

In [12]:
# Mari kita menerapkan beberapa persyaratan untuk memfilter data dan melihat jumlah baris dalam tabel yang difilter.

data_null = data.loc[(data['days_employed'].isna()) & (data['total_income'].isna())]

data_null.count()


children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

## Kesimpulan awal

[Apakah jumlah baris dalam tabel yang difilter sesuai dengan jumlah nilai yang hilang? Kesimpulan apa yang bisa kita buat dari hal ini?]

- tabel yang di hasilkan dari filter tidak spesifik untuk mengetahui jumlah nilai yang hilang, untuk itu membuat factor count

- Jumlah nilai yang hilang pada kolom days_employed & total_income saling beririsan dengan jumlah 2174 rows.

- Selanjutnya, untuk lebih memastikan, kita akan mengecek persentase jumlah baris atas nilai yang hilang terhadap keseluruhan data.

[Hitung persentase nilai yang hilang yang dibandingkan dengan seluruh kumpulan data. Apakah ini merupakan bagian data yang sangat besar? Jika demikian, Anda mungkin ingin mengisi nilai yang hilang. Untuk melakukannya, pertama-tama kita harus mempertimbangkan apakah data yang hilang bisa jadi disebabkan oleh karakteristik klien tertentu, seperti jenis pekerjaan atau yang lainnya. Anda perlu memutuskan karakteristik mana yang *Anda* pikir mungkin menjadi alasannya. Kedua, kita harus memeriksa apakah terdapat ketergantungan nilai yang hilang pada nilai indikator lain dengan kolom yang berisi karakteristik klien spesifik yang teridentifikasi.]



[Jelaskan langkah Anda selanjutnya dan bagaimana hubungannya dengan kesimpulan yang Anda buat sejauh ini.]

- Melakukan pemisahan data menggunakan isna(). hal ini di lakukan untuk dapat memberikan pengelihatan data yang berbeda dalam kasus ini float

In [13]:
# Mari kita memeriksa klien yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang

data.isna().sum() / len(data) * 100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

In [18]:
# Memeriksa distribusi 
# Membandingkan value antara data_null dan data asli
# Mencari unique value yang ada pada kolom di setiap kolom


categorical_columns = ['children','education','education_id', 'family_status', 
                       'family_status_id', 'gender', 'income_type','debt', ]

for col in categorical_columns:
    print ('\nUnique Values for %s in filtered data'%col)
    print (sorted(data_null[col].unique()))
    print('\nUnique Values for %s in fullset data'%col)
    print (sorted(data[col].unique()))
   



Unique Values for children in filtered data
[-1, 0, 1, 2, 3, 4, 5, 20]

Unique Values for children in fullset data
[-1, 0, 1, 2, 3, 4, 5, 20]

Unique Values for education in filtered data
["BACHELOR'S DEGREE", "Bachelor's Degree", 'PRIMARY EDUCATION', 'Primary Education', 'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education', 'Some College', "bachelor's degree", 'primary education', 'secondary education', 'some college']

Unique Values for education in fullset data
["BACHELOR'S DEGREE", "Bachelor's Degree", 'GRADUATE DEGREE', 'Graduate Degree', 'PRIMARY EDUCATION', 'Primary Education', 'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education', 'Some College', "bachelor's degree", 'graduate degree', 'primary education', 'secondary education', 'some college']

Unique Values for education_id in filtered data
[0, 1, 2, 3]

Unique Values for education_id in fullset data
[0, 1, 2, 3, 4]

Unique Values for family_status in filtered data
['civil partnership', 'divorced', 'married', 

### Kesimpulan awal 

- Terdapat berbagai cara untuk memerikas distribusi dari variabel data null 
- Data yang sudah di gilter hanya berisi data yang mempunyai value yang hilang atau nan
- Hanya ada perbedaan minor pada kolom education di data_null tidak memiliki value graduate degree dan pada kolom gender tidak memiliki value XNA.

**Kemungkinan penyebab hilangnya nilai dalam data**

[Ajukan ide-ide Anda tentang mengapa menurut Anda nilai-nilai tersebut kemungkinan hilang. Apakah menurut Anda mereka hilang secara acak atau terdapat pola?]

- absen yang tidak beraturan dan bonus dari setiap perusahan berbeda

[Mari kita mulai memeriksa apakah nilai hilang secara acak.]

# Pra Pemrosesan Data

In [12]:
# Memeriksa distribusi di seluruh dataset
# Menggunakan fungsi 
print ('\nUnique Values and distribution for education in data_null:')
print(data_null['education'].value_counts(normalize=True))
print ('\nUnique Values  and distribution for education in data:')
print(data['education'].value_counts(normalize=True))


Unique Values and distribution for education in data_null:
secondary education    0.647654
bachelor's degree      0.228151
SECONDARY EDUCATION    0.030819
Secondary Education    0.029899
some college           0.025299
Bachelor's Degree      0.011500
BACHELOR'S DEGREE      0.010580
primary education      0.008740
Some College           0.003220
SOME COLLEGE           0.003220
Primary Education      0.000460
PRIMARY EDUCATION      0.000460
Name: education, dtype: float64

Unique Values  and distribution for education in data:
secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
GRADUATE DEGREE        0.000046
Graduate Degree     

**Kesimpulan menengah**



- Terdapat data unik di [children, days_employed, total_income, purpose] biasanya di sebut outliner



# Periksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang


**Kesimpulan menengah**

- Terdapat berbagai outliner dimana hal ini terjadi karena berbagai macam input data yang berbeda beda bisa di lihat dari hasil kesimpulan di atas


# Memeriksa pola lainnya - jelaskan pola tersebut

- Terdapat data unik di [children, days_employed, total_income, purpose] biasanya di sebut outliner
- Seperti kesimpulan sebelumnya hal ini memberikan berbagai macam data unik yang dapat memberikan analisis tambahan untuk memberikan hipotesis baru

**Kesimpulan**


- Di butuhkan perbaikan dari beberapa tabel yang sudah saya jabarkan di atas



- NIlai yang hilang di atasi dengan membuat fungsi 



## Transformasi data


In [13]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
sorted(data['education'].unique())

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [14]:
print ('\nUnique Values and distribution for education in data_null:')
print(data_null['education'].value_counts())


Unique Values and distribution for education in data_null:
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64


In [15]:
# Perbaiki register jika diperlukan
data['education'] = data['education'].str.lower()

print(data['education'].unique())


["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [16]:
# Mengelompokan distribusi data yang tersebar di kolom tertentu

print ('\nUnique Values  and distribution for education in data:')
print(data['education'].value_counts())


Unique Values  and distribution for education in data:
secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


In [17]:
# Memeriksa semua nilai di kolom untuk memastikan kita telah memperbaikinya

data['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

[Periksa data kolom `children`]

In [20]:
# Mari kita lihat distribusi nilai pada kolom `children`

sorted(data['children'].unique())

[-1, 0, 1, 2, 3, 4, 5, 20]

In [21]:
# Menghitung jumlah value 20 dan -1 di column children

print(len(data.loc[(data['children'] == 20) | (data['children'] == -1)]))

123


In [26]:
# Menghitung presentase nilai unique di kolom children

len(data.loc[(data['children'] == 20) | (data['children'] == -1)]) / len(data) *100


0.5714285714285714

### Kesimpulan awal

- ada dua value yang bisa di bilang sangat berbeda -1 dan 20
- Kemungkinan besar ada kesalahan pada system


In [27]:
# Melakukan perbaikan pada kolum dengan cara re

data['children'] = data['children'].replace(-1,0).replace(20,0)


In [28]:
# Periksa kembali kolom `children` untuk memastikan semua telah diperbaiki

sorted(data['children'].unique())

[0, 1, 2, 3, 4, 5]

In [29]:
data['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

## Melakukan pengecekan di kolom days_employed

In [21]:
# Temukan data yang bermasalah di `days_employed`, jika terdapat masalah, dan hitung persentasenya

print(len(data.loc[(data['days_employed']< 0)]) / len(data) * 100)

print(len(data.loc[(data['days_employed']> 16790)]) / len(data) * 100)

73.89547038327527
16.004645760743323


[Jika jumlah data yang bermasalah tinggi, hal tersebut mungkin dikarenakan beberapa masalah teknis. Kami mungkin ingin mengusulkan alasan paling jelas mengapa hal tersebut dapat terjadi dan bagaimanakah kemungkinan data yang benar, karena kita tidak dapat menghapus baris yang bermasalah ini.]

In [31]:
# Atasi nilai yang bermasalah, jika ada

data['days_employed'] = data['days_employed'].abs()

data.loc[data['days_employed'] > 16790, 'days_employed'] = (data['dob_years'] - 19) * 365

In [23]:
# Periksa hasilnya - pastikan telah diperbaiki
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [34]:
print(data['days_employed'] / 365)

0        23.116912
1        11.026860
2        15.406637
3        11.300677
4        34.000000
           ...    
21520    12.409087
21521    48.000000
21522     5.789991
21523     8.527347
21524     5.437007
Name: days_employed, Length: 21525, dtype: float64


Di lakukan pengecekan lanjutan terhadap kolom days employed untu mengetahui nilai nilai yang mencirigakan karena kesalahan system

In [35]:
# hitung persentase yang bermasalah

len(data.loc[(data['days_employed'] < 0)]) / len(data) * 100


0.0

In [24]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya

data['dob_years'].value_counts()/ len(data) * 100


35    2.866434
40    2.829268
41    2.819977
34    2.801394
38    2.778165
42    2.773519
33    2.699187
39    2.662021
31    2.601626
36    2.578397
44    2.541231
29    2.531940
30    2.508711
48    2.499419
37    2.494774
50    2.387921
43    2.383275
32    2.369338
49    2.360046
28    2.336818
45    2.308943
27    2.290360
56    2.262485
52    2.248548
47    2.229965
54    2.225319
46    2.206736
58    2.141696
57    2.137050
53    2.132404
51    2.081301
59    2.062718
55    2.058072
26    1.895470
60    1.751452
25    1.658537
61    1.649245
62    1.635308
63    1.249710
64    1.231127
24    1.226481
23    1.180023
65    0.901278
66    0.850174
22    0.850174
67    0.775842
21    0.515679
0     0.469222
68    0.459930
69    0.394890
70    0.301974
71    0.269454
20    0.236934
72    0.153310
19    0.065041
73    0.037166
74    0.027875
75    0.004646
Name: dob_years, dtype: float64

[Putuskan apa yang akan Anda lakukan dengan nilai yang bermasalah dan jelaskan alasannya.]

Terdapat data pada [dob_years] berumur 0 tahun pada dengan presentase 0,4%. nilai ini bisa di bilang merupakan keganjilan pada table yang sebaiknya tidak di ubah menjadi [NaN] dan jika di lingkungan kerja bagus untuk mengkonsultasikan terlebih dahulu dengan atasan 

In [37]:
# Atasi masalah pada kolom `dob_years`, jika terdapat masalah

data.loc[(data['dob_years'] == 0), 'dob_years'] = data['dob_years'].median()

In [39]:
# Periksa hasilnya - pastikan telah diperbaiki
data['dob_years'].value_counts().sort_values()

75      1
74      6
73      8
19     14
72     33
20     51
71     58
70     65
69     85
68     99
21    111
67    167
66    183
22    183
65    194
23    254
24    264
64    265
63    269
62    352
61    355
25    357
60    377
26    408
55    443
59    444
51    448
53    459
57    460
58    461
46    475
54    479
47    480
52    484
56    487
27    493
45    497
28    503
49    508
32    510
43    513
50    514
37    537
48    538
30    540
29    545
44    547
36    555
31    560
39    573
33    581
38    598
34    603
41    607
40    609
35    617
42    698
Name: dob_years, dtype: int64

## Pengecekan kolom di family _status

In [27]:
# Mari kita lihat nilai untuk kolom

data['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

### Kesimpulan awal
- Distribusi persebaran value object data di dalam kolum merata dan tidak ada data yang mencurigakan

### Melakukan pemerikasan pada kolom Gender

In [30]:
# Mari kita melihat nilainya di kolom

data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [43]:
# Atasi nilai-nilai yang bermasalah, jika ada

data.loc[data['gender'] == 'nan', 'gender'] = 'F'
data.loc[data['gender'] == 'XNA', 'gender'] = 'unknown'

In [44]:
# Periksa hasilnya - pastikan telah diperbaiki

sorted(data.gender.unique())

['F', 'M', 'unknown']

In [45]:
data['gender'].value_counts()

F          14236
M           7288
unknown        1
Name: gender, dtype: int64

### Melakukan kolom income_type

In [46]:
# Mari kita lihat nilai dalam kolom

data['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

## Melakukan pemeriksaan nilai nilai duplikat
- Dilihat dari distribusi data yang sepertinya merata, bisa di simpulkan tidak ada masalah

In [34]:
# Periksa hasilnya - pastikan telah diperbaiki
data['income_type'].duplicated().sum()

21517

[Sekarang saatnya melihat apakah terdapat duplikasi di dalam data kita. Jika kita menemukannya, Anda harus memutuskan apa yang akan Anda lakukan dengan duplikat tersebut dan menjelaskan alasannya.]

In [35]:
# Memeriksa duplikat

data.duplicated().sum()

72

In [36]:
# Atasi duplikat, jika ada

data = data.drop_duplicates().reset_index(drop=True)

In [37]:
# Terakhir periksa apakah kita memiliki duplikat
data.duplicated().sum()

0

In [38]:
# Periksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang Anda lakukan

data.shape

(21453, 12)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [48]:
# Persentase perubahan kolom days_employed setelah manipulasi pertama diatas 
(19419 - 19351) / 19351

0.0035140302826727303

### Kesimpulan awal 
- Days employed memiliki nilai yang janggal dimana terdapat 0.35 % perubahan value setelah di lakukan pemeriksaan nilai nilai yang hilang

# Bekerja dengan nilai yang hilang

[Untuk mempercepat pekerjaan dengan beberapa data, Anda mungkin ingin menggunakan dictionary untuk beberapa nilai, di mana tersedia ID. Jelaskan mengapa dan dictionary apakah yang akan Anda gunakan.]

In [39]:
# Temukan dictionary

sorted(data.dob_years.unique())
       

[19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0]

### Memperbaiki nilai yang hilang di `total_income`

In [51]:
# Mari menulis fungsi untuk menghitung kategori usia
def age_group(age):

    if age <= 19:
        return 'Siap Kerja'
    if age >= 22:
        return 'Sudah Kerja'
    if age <= 64:
        return 'Eksekutif Kerja'
    return 'Pensiun'


    

<div class="alert alert-warning">
<b>Reviewer's comment v1</b> <a class="tocSkip"></a>

Untuk menghindari subjektifitas dan tidak adanya referensi yang digunakan dalam pengelompokan, kategori dapat dinamai sesuai dengan batas atas dan bawah pada tiap kelompok misa, `sampai 24 tahun` dan `64 tahun keatas`.

</div>

In [52]:
# Lakukan pengujian apakah fungsi bekerja atau tidak
age_group(55)

'Sudah Kerja'

In [53]:
# Membuat kolom baru berdasarkan fungsi

data['age_group'] = data['dob_years'].apply(age_group)

In [54]:
# Memeriksa bagaimana nilai di dalam kolom baru

data['age_group'].value_counts()

Sudah Kerja        21349
Eksekutif Kerja      162
Siap Kerja            14
Name: age_group, dtype: int64

In [44]:
# Membuat tabel tanpa nilai yang hilang dan menampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
# exclude missing value data

data_clean = data.loc[~(data['total_income'].isna()) & ~(data['income_type'].isna())]

data_clean.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja
4,0,12410.000000,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja


In [45]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah Anda identifikasi

data.pivot_table(index='income_type', values='total_income', aggfunc='mean')


,total_income
income_type,
business,32386.793835
civil servant,27343.729582
employee,25820.841683
entrepreneur,79866.103000
paternity / maternity leave,8612.661000
retiree,21940.394503
student,15712.260000
unemployed,21014.360500


In [46]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah Anda identifikasi

data.pivot_table(index='income_type', values='total_income', aggfunc='median')

,total_income
income_type,
business,27577.2720
civil servant,24071.6695
employee,22815.1035
entrepreneur,79866.1030
paternity / maternity leave,8612.6610
retiree,18962.3180
student,15712.2600
unemployed,21014.3605


Kita akan memakai median untuk mengganti nilai yang hilang karena income setiap orang bervariasi, jika menggunakan mean dirasa kurang tepat untuk merepresentasikan nilai yang hilang.

In [58]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang

rules = data['income_type'].unique()

def data_imputation(data, column_filter, column_selected, column_target, rules): #paramater
    data[column_target] = data[column_selected]
    for rule in rules:
        # get median
        median = data.loc[(data[column_filter]==rule) & ~(data[column_target].isna()), column_target].median()
        # change missing value
        data.loc[(data[column_filter]==rule) & (data[column_target].isna()), column_target] = median
    return data
 


In [59]:
# Memeriksa bagaimana nilai di dalam kolom baru

data_imputation(data, column_filter='income_type', column_selected='total_income',column_target='total_income_clean', rules=rules)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,Sudah Kerja,35966.698
21521,0,17520.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,Sudah Kerja,24959.969
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,Sudah Kerja,14347.610
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,Sudah Kerja,39054.888


In [60]:
# Terapkan fungsi ke setiap baris
# Terapkan fungsi ke income_type
 

data_imputation(data, column_filter='income_type', column_selected='total_income',column_target='total_income', rules=rules)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,Sudah Kerja,35966.698
21521,0,17520.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,Sudah Kerja,24959.969
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,Sudah Kerja,14347.610
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,Sudah Kerja,39054.888


In [61]:
# Periksa apakah kita mendapatkan kesalahan
data['total_income_clean'].isna().sum()

0

In [62]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572


[Jika Anda menemukan kesalahan dalam menyiapkan nilai data yang hilang, artinya mungkin terdapat sesuatu yang khusus terkait dengan data untuk kategori tersebut. Mari pikirkan - Anda mungkin ingin memperbaiki beberapa hal secara manual, jika terdapat cukup data untuk menemukan median/rata-rata.]


In [51]:
# Mengganti nilai yang hilang jika terdapat kesalahan
sorted(data['total_income'].unique())

# Tidak Terdapat Kesalahan di lihat dari hasil koding

[3306.762,
 3392.845,
 3418.824,
 3471.216,
 3503.298,
 3595.641,
 3815.153,
 3913.227,
 4036.463,
 4049.374,
 4212.77,
 4245.348,
 4386.4,
 4444.179,
 4465.254,
 4494.861,
 4592.45,
 4650.812,
 4664.644,
 4672.012,
 4708.271,
 4759.97,
 4812.103,
 4818.545999999999,
 4860.001,
 4919.749,
 5002.295,
 5028.623,
 5029.439,
 5037.321,
 5045.56,
 5053.838,
 5090.55,
 5112.186,
 5137.573,
 5148.514,
 5167.9940000000015,
 5168.082,
 5172.669,
 5195.285,
 5208.353,
 5217.0340000000015,
 5220.544,
 5259.254,
 5274.611,
 5288.165,
 5290.465,
 5330.769,
 5331.621,
 5335.014,
 5402.85,
 5409.738,
 5430.683000000001,
 5443.908,
 5452.4940000000015,
 5461.996,
 5464.092,
 5478.583000000001,
 5490.018,
 5496.834,
 5514.581,
 5515.539000000002,
 5529.334,
 5531.2040000000015,
 5562.874,
 5577.521,
 5579.965,
 5591.44,
 5595.911999999998,
 5604.991999999998,
 5622.0790000000015,
 5630.865,
 5639.846,
 5651.584,
 5703.853,
 5768.392,
 5772.8780000000015,
 5801.651,
 5803.271,
 5820.374,
 5826.733,
 583

[Ketika Anda berpikir Anda telah selesai dengan `total_income`, periksa apakah jumlah total nilai di kolom ini sesuai dengan jumlah nilai di kolom lain.]

In [52]:
# Memeriksa jumlah entri di kolom

data.count()

# Tidak Terdapat Kesalahan di lihat dari hasil koding

children              21453
days_employed         19351
dob_years             21453
education             21453
education_id          21453
family_status         21453
family_status_id      21453
gender                21453
income_type           21453
debt                  21453
total_income          21453
purpose               21453
age_group             21453
total_income_clean    21453
dtype: int64

In [63]:
data['total_income_clean'].isna().sum()

0

###  Memperbaiki nilai di `days_employed`

In [53]:
# Distribusi median dari `days_employed` berdasarkan parameter yang Anda identifikasi

data.pivot_table(index='age_group', values='days_employed', aggfunc='median')

,days_employed
age_group,
Eksekutif Kerja,627.368203
Siap Kerja,724.492610
Sudah Kerja,2214.576518


In [54]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang Anda identifikasi
data.pivot_table(index='age_group', values='days_employed', aggfunc='mean')

,days_employed
age_group,
Eksekutif Kerja,701.334134
Siap Kerja,633.678086
Sudah Kerja,4571.589856


Kita memilih menggunakan median untuk mengganti nilai yang hilang pad kolom days_employed karena value yang bervariasi pada kolom days_employed tidak memungkinkan memakai mean.

In [64]:
# Mari tulis fungsi yang menghitung rata-rata atau median (tergantung keputusan Anda) berdasarkan parameter yang Anda identifikasi

rules = data['age_group'].unique()

def data_imputation(data, column_filter, column_selected, rules):
    for rule in rules:
        # get median
        median = data.loc[(data[column_filter]==rule) & ~(data[column_selected].isna()), column_selected].median()
        # change missing value
        data.loc[(data[column_filter]==rule) & (data[column_selected].isna()), column_selected] = median
    return data 



In [65]:
# Periksa bahwa fungsi bekerja

data_imputation(data = data, column_filter='age_group', column_selected='days_employed', rules=rules)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,Sudah Kerja,35966.698
21521,0,17520.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,Sudah Kerja,24959.969
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,Sudah Kerja,14347.610
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,Sudah Kerja,39054.888


In [57]:
# Periksa bahwa fungsi bekerja
data['days_employed'].isna().sum()


0

In [59]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang
data.count()

children              21453
days_employed         21453
dob_years             21453
education             21453
education_id          21453
family_status         21453
family_status_id      21453
gender                21453
income_type           21453
debt                  21453
total_income          21453
purpose               21453
age_group             21453
total_income_clean    21453
dtype: int64

## Analisis Data Eksploratif

Pada bagian Hipotesis kita akan menjawab pertanyaan-pertanyaan berikut ini:

1.Apakah terdapat hubungan antara memiliki anak dan probabilitas seseorang melakukan gagal bayar pinjaman?

2.Apakah terdapat hubungan antara status perkawinan dan probabilitas seseorang melakukan gagal bayar pinjaman?

3.Apakah terdapat hubungan antara tingkat pendapatan dan probabilitas seseorang melakukan gagal bayar pinjaman?

4.Bagaimana perbedaan tujuan pinjaman memengaruhi probabilitas seseorang melakukan gagal bayar pinjaman?

In [69]:
# Tampilkan nilai data yang Anda pilih untuk pengkategorian

categorical_column = ['gender', 'children', 'education', 'debt']

[Let's check unique values]

In [61]:
# Periksa nilai unik

data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [62]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [63]:
data['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [70]:
sorted(data['education'].unique())

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [64]:
data['debt'].unique()

array([0, 1])

[Kelompok utama apakah yang dapat Anda identifikasi berdasarkan nilai uniknya?]

- Kelompok utama untuk memeriksa nilai unik yang menggunakan['days_employed]

[Berdasarkan topik ini, kita ingin mengkategorikan data kita.]


In [72]:
# Mari kita menulis fungsi untuk mengkategorikan data berdasarkan topik umum

value= data['debt']

def debt_probability(value):
    if value == 1:
        return 'failed'
    else:
        return 'pass'


In [73]:
# Buat kolom dengan kategori dan hitung nilainya

data['debt_probability'] = data['days_employed'].apply(debt_probability)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_probability
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102,pass
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802,pass
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752,pass
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568,pass
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572,pass


## Pengkategorian Data

In [74]:
# Lihat melalui semua data numerik di kolom yang Anda pilih untuk pengkategorian

selected_category = ['education','family_status','debt_probability']

selected_num_columns = ['days_employed','dob_years','total_income']

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_probability
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102,pass
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802,pass
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752,pass
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568,pass
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572,pass


In [68]:
# Mendapatkan kesimpulan statistik untuk kolomnya

data[selected_num_columns].describe()

,days_employed,dob_years,total_income
count,21453.000000,21453.000000,21453.000000
mean,4311.411964,43.469025,26451.382421
std,5059.112470,12.214162,15710.314734
min,-6935.000000,19.000000,3306.762000
25%,1016.929210,33.000000,17219.352000
50%,2214.576518,42.000000,22815.103500
75%,4769.746242,53.000000,31331.461000
max,20075.000000,75.000000,362496.645000


Untuk memberikan variable perhitungan dalam analysis yang di lakukan 

In [76]:
# Membuat fungsi untuk pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang
value = data['total_income']

def categorical_total_income(value):
    if value <= 11234:
        return 'poor'
    elif value <= 17983:
        return 'medium wealty'
    else:
        return 'rich'

In [77]:
# Lakukan pengujian apakah fungsi bekerja atau tidak
print(categorical_total_income(15))


poor


In [78]:
# Membuat kolom dengan kategori


data['categorical_total_income'] = data['days_employed'].apply(categorical_total_income)

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_probability,categorical_total_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102,pass,poor
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802,pass,poor
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752,pass,poor
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568,pass,poor
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572,pass,medium wealty
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,Sudah Kerja,40922.170,pass,poor
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,Sudah Kerja,38484.156,pass,poor
7,0,152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education,Sudah Kerja,21731.829,pass,poor
8,2,6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding,Sudah Kerja,15337.093,pass,poor
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,Sudah Kerja,23108.150,pass,poor


In [79]:
# Menghitung setiap nilai kategori untuk melihat pendistribusian
data['categorical_total_income'].value_counts()

poor             18106
medium wealty     3205
rich               214
Name: categorical_total_income, dtype: int64

In [80]:
# membuat kategori dari kolom purpose
# Mari kita menulis fungsi untuk mengkategorikan data berdasarkan topik umum
# kategori untuk kolom purpose karena nilai unik pada kolom purpose >10 dan beberapa value mengandung arti yang sama
# kelompok kategori dibuat ke kolom baru

data.loc[data['purpose'].isin(['real estate transactions', 'purchase of my own house', 'transactions with my real estate', 'housing', 'purchase of the house', 'housing transactions', 'transactions with commercial real estate', 'purchase of the house for my family', 'buy real estate', 'buy commercial real estate', 'buy residential real estate']), 'purpose_group'] = 'buy house'
data.loc[data['purpose'].isin(['wedding ceremony', 'to have a wedding', 'having a wedding']), 'purpose_group'] = 'wedding'
data.loc[data['purpose'].isin(['to become educated', 'profile education', 'getting higher education', 'going to university', 'university education', 'getting an education', 'supplementary education', 'to get a supplementary education', 'education']), 'purpose_group'] = 'education'
data.loc[data['purpose'].isin(['construction of own property', 'housing renovation', 'building a real estate']), 'purpose_group'] = 'housing renovation'
data.loc[data['purpose'].isin(['buying property for renting out', 'building a property', 'property']), 'purpose_group'] = 'buy property'
data.loc[data['purpose'].isin(['to buy a car', 'to own a car', 'second-hand car purchase', 'purchase of a car', 'buying my own car', 'buying a second-hand car', 'cars', 'car', 'car purchase']), 'purpose_group'] = 'car purchase'

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_probability,categorical_total_income,purpose_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,Sudah Kerja,40620.102,pass,poor,buy house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802,pass,poor,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,Sudah Kerja,23341.752,pass,poor,buy house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,Sudah Kerja,42820.568,pass,poor,education
4,0,12410.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,Sudah Kerja,25378.572,pass,medium wealty,wedding


In [83]:
# Memeriksa fungsi bekerja untuk salah satu kategori di kolom purpose

data.loc[data['purpose'] == 'car purchase'].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_clean,debt_probability,categorical_total_income,purpose_group
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,Sudah Kerja,17932.802,pass,poor,car purchase
13,0,1846.641941,54,some college,2,married,0,F,employee,0,20873.317,car purchase,Sudah Kerja,20873.317,pass,poor,car purchase
21,1,253.685166,33,secondary education,1,civil partnership,1,F,employee,0,21089.953,car purchase,Sudah Kerja,21089.953,pass,poor,car purchase
37,0,6448.810860,43,BACHELOR'S DEGREE,0,civil partnership,1,F,business,0,24660.621,car purchase,Sudah Kerja,24660.621,pass,poor,car purchase
43,0,4375.681384,43,some college,2,divorced,3,F,business,0,43673.141,car purchase,Sudah Kerja,43673.141,pass,poor,car purchase


# Memeriksa Hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan membayar kembali tepat waktu?**

In [84]:
# Periksa data anak dan membayar kembali dengan tepat waktu

data['children'].value_counts() 


0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [85]:
# Menghitung tarif otomatis berdasarkan jumlah anak

children_debt = data.pivot_table(index='children', columns= 'debt', values= 'debt_probability', aggfunc= 'count')
children_debt['percent'] = children_debt[1]/ (children_debt[0] + children_debt[1]) * 100
children_debt

debt,0,1,percent
children,,,
0,13200.0,1072.0,7.511211
1,4374.0,444.0,9.215442
2,1861.0,194.0,9.440389
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Kesimpulan**

[Tulis kesimpulan Anda berdasarkan manipulasi dan pengamatan Anda.]

- Semakin banyak anak semakin besar presentase untuk melunasi hutang 


**Apakah terdapat korelasi antara status keluarga dengan membayar kembali tepat waktu?**

In [86]:
# Periksa data status keluarga dan membayar kembali dengan tepat waktu

data[['family_status','debt']] 


,family_status,debt
0,married,0
1,married,0
2,married,0
3,married,0
4,civil partnership,0
...,...,...
21520,civil partnership,0
21521,married,0
21522,civil partnership,1
21523,married,1


In [87]:
family_debt = data.pivot_table(index='family_status', columns= 'debt', values= 'debt_probability', aggfunc= 'count')
family_debt['percent'] = family_debt[1]/ (family_debt[0] + family_debt[1]) * 100
family_debt

debt,0,1,percent
family_status,,,
civil partnership,3789,388,9.288963
divorced,1110,85,7.112971
married,11449,931,7.520194
unmarried,2539,274,9.740491
widow / widower,897,63,6.562500


**Kesimpulan**

[Tulis kesimpulan Anda berdasarkan manipulasi dan pengamatan Anda.]

- Yang berstatus janda memiliki kemampuan bayar hutang lebih vesar dari pada family_status yang lain 

**Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?**

In [88]:
data[['total_income','categorical_total_income','debt']]

,total_income,categorical_total_income,debt
0,40620.102,poor,0
1,17932.802,poor,0
2,23341.752,poor,0
3,42820.568,poor,0
4,25378.572,medium wealty,0
...,...,...,...
21520,35966.698,poor,0
21521,24959.969,medium wealty,0
21522,14347.610,poor,1
21523,39054.888,poor,1


In [89]:
# Menghitung tarif otomatis berdasarkan tingkat pendapatan

income_debt = data.pivot_table(index='categorical_total_income', columns= 'debt', values= 'debt_probability', aggfunc= 'count')
income_debt['percent'] = income_debt[1]/ (income_debt[0] + income_debt[1]) * 100
income_debt

debt,0,1,percent
categorical_total_income,,,
medium wealty,3040,165,5.148206
poor,16536,1570,8.671159
rich,208,6,2.803738


**Kesimpulan**

[Tulis kesimpulan Anda berdasarkan manipulasi dan pengamatan Anda.]

- Dari pembuatan analisis [income_categorical] bisa membuat analyis baru untuk pembayaran kredit dengan paramater dari kolom ini 

**Bagaimana tujuan kredit memengaruhi tarif otomatis?**

In [80]:
# Periksa persentase tarif otomatis untuk setiap tujuan kredit dan lakukan penganalisisan

print(data[['purpose_group', 'debt','debt_probability']])

      purpose_group  debt debt_probability
0         buy house     0             pass
1      car purchase     0             pass
2         buy house     0             pass
3         education     0             pass
4           wedding     0             pass
...             ...   ...              ...
21448     buy house     0             pass
21449  car purchase     0             pass
21450  buy property     1             pass
21451  car purchase     1             pass
21452  car purchase     0             pass

[21453 rows x 3 columns]


In [90]:
# Periksa persentase tarif otomatis untuk setiap tujuan kredit dan lakukan penganalisisan

purpose_debt = data.pivot_table(index='purpose_group', columns= 'debt', values= 'debt_probability', aggfunc= 'count')
purpose_debt['percent'] = purpose_debt[1]/ (purpose_debt[0] + purpose_debt[1]) * 100
purpose_debt


debt,0,1,percent
purpose_group,,,
buy house,6551,509,7.209632
buy property,1759,148,7.760881
car purchase,3912,403,9.339513
education,3652,370,9.199403
housing renovation,1748,125,6.673785
wedding,2162,186,7.921635


**Kesimpulan**



- Dari kesimpulan dengan menggunakan [value_counts] dari table [purpose] fungsi tersebut memperlihatkan pemberian kredit. dimana presentasi paling banyak adalah pembelian mobil second

- Pembelian mobil second ada di atas 15 %

- properti juga merupakan sektor yang banyak mendapatkan hasil dari pemberian kredit melebihi otomotif

- begitu pula dengan pendidikan melebihi properti



# Kesimpulan Umum 


- Dalam memberikan evaluasi kredit sebuah perusahaan untuk bisa memberikan kesimpulan apakah bisa memberikan pinjaman atau tidak di butuhkan evaluasi secara mendalam.

- hal pertama yang bisa di lakukan adalah melihat data yang ada

- mengetahui tipe data yang di dapatkan

- melakukan pengecekan terhadap data yang ada dengan mengetahui nilai nilai yang unique dalam project ini data float yang mempunyai 'NaN'

- outliner di [days_employed] banyak memiliki nilai minus
    - Apa yang harus di lakukan jika memiliki nilai minus dalam kasus dalam perusahaan yang sebenarnyajika data tidak bisa di lakukan pengecekan langsung dari sumbernya?
    
- Kategori pendidikan yang memiliki value dengan penulisan berbeda beda mungkin bisa jadi masalah oleh karena itu di butuhkan perbaikan

- [Gender] memiliki masalah yang harus di perbaiki dikarenakan ada jenis data yang salah 

- [days_employed] dan [total_income] memiliki menjadi bermasalah karena membutuhkan kategori di tambahan dalam kolom untuk di jadikan sebagai paramater untuk memudahkan menganalisis data 

    - dalam project ini [age_group,total_income_clean,debt_probability,income_categorical]
    
    - ada beberapa penambahan kolom dan tidak bisa di hapus karena walaupun fungsi [def] sudah di ganti dengan fungsi baru. tetap muncul di fungsi koding baru di tabel nya ? apakah bisa menghapus kolom ? apakah kolom bisa di pindahkan ?

- Pengkategorian data di lakukan untuk bisa memberikan kemudahan dalam memilah data yang sudah di perbaiki dan membuat kategori baru untuk bisa memberikan pinjaman

- memeriksa hipotesis untuk dapat memberikan gambaran pemberian kredit yang bermasalah atau tidak

    - presentase bisa dilihat untuk dapat memberikan sebuah kemudahan dalam analysis data yang sudah di perbaiki
    
- dari hipotesis bisa memberikan analysis baru yang bisa di olah lagi untuk kredit baru dan berkelanjutan 
